In [24]:
# -*- coding: utf-8 -*-
# import essential packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shap

from sklearn.feature_selection import VarianceThreshold, SelectFromModel, chi2, SelectKBest, f_classif
from collections import Counter
from sklearn.model_selection import StratifiedKFold, GridSearchCV,cross_validate, cross_val_predict, train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier,GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, make_scorer, cohen_kappa_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTENC, SMOTE
from pathlib import Path

In [25]:
# read raw data and clean the data by replacing NAN with 0
data_raw = pd.read_csv('../raw_data/final_inputed_results_withChemicals.csv', header=[0])
data_raw = data_raw.fillna(0)
data_raw

,id_of_patients,group,gender,age,BMI,TNM_stage,tumor_location,family_cancer_history,education,smoking_history,...,Se,Rb,Sr,Ba,Pb,Mg,Li,Na,Si,Ca
0,sx-EC-P1,1,1,70,30.836531,1,2,1,3,0,...,0.089323,0.249740,0.064062,0.016146,0.010156,24.649566,0.017535,3749.770920,0.803561,105.182812
1,sx-EC-P2,1,1,70,23.183391,4,2,0,1,1,...,0.184365,0.385241,0.059287,0.007004,0.011257,28.444861,0.014092,3674.550177,1.211007,111.660538
2,sx-EC-P3,1,1,76,27.681661,2,3,0,2,1,...,0.104590,0.320784,0.079835,0.004538,0.049097,29.545608,0.004625,3.157379,0.822846,0.156441
3,sx-EC-P4,1,2,65,24.034610,3,2,0,2,0,...,0.114020,0.261976,0.066218,0.004553,0.001035,24.609071,0.010623,2787.701259,1.001759,88.843870
4,sx-EC-P5,1,2,70,26.171875,2,2,0,1,0,...,0.090681,0.384402,0.060542,0.006345,0.002644,26.429698,0.017537,4658.693545,1.600000,124.027231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,bj-EC-P36,0,1,58,23.938990,3,2,1,3,1,...,0.292232,0.451276,0.040196,0.001304,0.000281,33.455586,0.003549,4485.171392,4.996415,138.829766
97,bj-EC-P37,0,1,60,19.257029,1,2,0,2,1,...,0.087347,0.459088,0.031085,0.004367,0.000332,26.752045,0.003682,3809.909142,13.059216,112.164933
98,bj-EC-P38,0,1,59,19.607157,2,3,0,2,1,...,0.106808,0.496948,0.038732,0.003990,0.000303,30.051487,0.003599,4852.086228,4.545775,140.200469
99,bj-EC-P39,0,1,42,21.433627,1,3,0,2,0,...,0.068167,0.431190,0.012862,0.005466,0.000415,28.441908,0.007503,4770.198821,12.063987,138.368167


In [26]:
# read group information from processed raw data
group = data_raw["group"]
group

0      1
1      1
2      1
3      1
4      1
      ..
96     0
97     0
98     0
99     0
100    0
Name: group, Length: 101, dtype: int64

In [27]:
# read demography information from processed raw data
demography = data_raw[["gender", "age", "BMI", "education"]]
demography

,gender,age,BMI,education
0,1,70,30.836531,3
1,1,70,23.183391,1
2,1,76,27.681661,2
3,2,65,24.034610,2
4,2,70,26.171875,1
...,...,...,...,...
96,1,58,23.938990,3
97,1,60,19.257029,2
98,1,59,19.607157,2
99,1,42,21.433627,2


In [28]:
# read dietary and behavioral habits information from processed raw data
dietary_and_behavioral_habits = data_raw[["smoking_history", "drinking_habit", "green_tea_habit", "regular_staple_food", 
                                      "frenquency_on_pickled_food", "frenquency_on_fresh_fruit_vegetable", 
                                      "preference_of_hot_food_and_water", "preference_of_hard_food", "daily_meal_times"]]
dietary_and_behavioral_habits

,smoking_history,drinking_habit,green_tea_habit,regular_staple_food,frenquency_on_pickled_food,frenquency_on_fresh_fruit_vegetable,preference_of_hot_food_and_water,preference_of_hard_food,daily_meal_times
0,0,1,1,1,2,2,1,0,2
1,1,1,0,6,1,2,1,1,1
2,1,1,1,1,3,2,1,1,1
3,0,0,0,6,2,3,1,1,2
4,0,0,0,6,3,1,0,0,2
...,...,...,...,...,...,...,...,...,...
96,1,1,1,6,1,1,1,1,3
97,1,1,1,1,3,2,1,0,2
98,1,1,1,2,1,1,1,0,3
99,0,1,0,6,2,1,1,1,2


In [29]:
# read clinic biochemical index information from processed raw data
clinic_biochemical_index = data_raw[["LYM_pcent", "MON_pcent", "NEU_pcent", "GLB", "ALT_/AST", "ALP", "γ_GGT", "UREA", "CRE"]]
clinic_biochemical_index

,LYM_pcent,MON_pcent,NEU_pcent,GLB,ALT_/AST,ALP,γ_GGT,UREA,CRE
0,18.5,3.9,77.4,28.6,0.83,87.0,23.00,3.86,70.0
1,20.4,6.6,69.9,26.3,0.69,61.0,24.52,5.73,76.0
2,25.3,6.6,60.4,33.6,0.64,96.0,13.00,6.26,59.0
3,37.0,8.4,47.8,26.4,0.53,63.0,21.00,4.16,51.0
4,31.6,7.8,57.6,25.2,0.50,67.0,13.00,7.67,76.0
...,...,...,...,...,...,...,...,...,...
96,19.0,6.8,72.3,32.6,1.73,63.0,49.00,5.69,65.0
97,25.2,4.9,69.0,23.9,0.91,76.0,22.00,5.87,92.0
98,31.3,6.8,58.3,26.0,1.27,86.0,96.00,8.30,71.0
99,29.2,5.5,63.6,27.8,1.38,62.0,19.00,7.68,66.0


In [30]:
# read chemical exposome from processed raw data
chemical_exposome = data_raw.iloc[:,28:] # from column AC (namely Dodecanoic acid) to the end
chemical_exposome

,Dodecanoic acid,3-Hydroxydecanoic acid,Cortisone,Benzaldehyde,Caryophyllene alpha-oxide,Alpha-dimorphecolic acid,Androsterone sulfate,Dehydroepiandrosterone sulfate,Mesylate,Sphingosine 1-phosphate,...,Se,Rb,Sr,Ba,Pb,Mg,Li,Na,Si,Ca
0,1.075559e+08,2217096.927,336473.91010,1.196122e+06,8664196.613,3444225.406,5.246442e+07,1.481620e+08,1.571274e+07,23530863.69,...,0.089323,0.249740,0.064062,0.016146,0.010156,24.649566,0.017535,3749.770920,0.803561,105.182812
1,2.769898e+08,8232623.720,739152.90820,9.265912e+05,7156456.223,6119370.366,3.851976e+07,1.174557e+08,1.749723e+07,18826778.24,...,0.184365,0.385241,0.059287,0.007004,0.011257,28.444861,0.014092,3674.550177,1.211007,111.660538
2,9.816326e+07,2961288.561,790557.30130,1.178029e+06,2387112.445,2337909.206,4.092974e+07,5.300348e+07,2.254973e+07,23535272.17,...,0.104590,0.320784,0.079835,0.004538,0.049097,29.545608,0.004625,3.157379,0.822846,0.156441
3,1.584675e+08,4623278.303,117876.85330,9.926589e+05,8918536.766,4343820.774,4.542826e+07,2.924513e+07,3.371170e+07,24197693.14,...,0.114020,0.261976,0.066218,0.004553,0.001035,24.609071,0.010623,2787.701259,1.001759,88.843870
4,7.416078e+07,2333854.010,547141.84680,9.200433e+05,5539531.547,1556370.799,9.027555e+06,2.025362e+07,1.232532e+07,21751485.40,...,0.090681,0.384402,0.060542,0.006345,0.002644,26.429698,0.017537,4658.693545,1.600000,124.027231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,8.752311e+07,6107999.109,82403.86520,6.298567e+05,3186965.440,2850667.379,5.069764e+07,1.287732e+08,1.084380e+07,10341660.80,...,0.292232,0.451276,0.040196,0.001304,0.000281,33.455586,0.003549,4485.171392,4.996415,138.829766
97,6.033596e+07,3280569.645,17993.81052,6.618920e+05,6329133.962,2691711.072,4.958056e+07,8.312855e+07,1.551989e+07,10506455.66,...,0.087347,0.459088,0.031085,0.004367,0.000332,26.752045,0.003682,3809.909142,13.059216,112.164933
98,6.195672e+07,2914028.339,86779.43696,5.806832e+05,6274758.088,3214817.250,4.917949e+07,7.325360e+07,8.334815e+06,14616223.93,...,0.106808,0.496948,0.038732,0.003990,0.000303,30.051487,0.003599,4852.086228,4.545775,140.200469
99,5.878176e+07,2912714.059,113696.23930,5.931401e+05,7328779.005,3318208.832,5.416800e+07,8.111983e+07,1.145050e+07,15472194.78,...,0.068167,0.431190,0.012862,0.005466,0.000415,28.441908,0.007503,4770.198821,12.063987,138.368167


In [31]:
medication_history = data_raw[["medication_history"]]
medication_history

,medication_history
0,0
1,0
2,1
3,1
4,0
...,...
96,1
97,1
98,0
99,0


In [32]:

data = clinic_biochemical_index

data = pd.concat([dietary_and_behavioral_habits, demography], axis=1)

data = pd.concat([medication_history, dietary_and_behavioral_habits, demography], axis=1)

data = data_raw.drop(columns=["id_of_patients","group"])

# data = chemical_exposome
target = group

# Remove low variance:
print("Before removing low variance:{}".format(data.shape))
selector = VarianceThreshold(threshold=0.05)
selector.fit_transform(data)
cols=selector.get_support(indices=True)
X = data.iloc[:,cols]
print("After removing low variance:{}".format(data.shape))

# Preprocessing - Feature Selection
std_scaler = MinMaxScaler()
X = std_scaler.fit_transform(X)

_, pvalue = chi2(X, target)
threshold = 0.05
cols_model = np.where(pvalue < threshold)[0]
coef_sel = pvalue[cols_model]
print(len(cols_model), X.shape)
print(cols_model)

X = X[:, cols_model]

Before removing low variance:(101, 341)
After removing low variance:(101, 341)
20 (101, 333)
[  8   9  12  21  34  54  66 134 148 155 184 191 194 217 241 243 270 306
 328 331]


In [33]:
select_val = data.columns.values[cols_model]
# print(cols_model)
data = pd.DataFrame(X, columns=select_val)
data

,drinking_habit,green_tea_habit,frenquency_on_fresh_fruit_vegetable,ALT_/AST,Mesylate,Thromboxane B3,3-Carboxy-4-methyl-5-propyl-2-furanpropionic acid,Dibutyl phthalate,ACar(15:0),ACar(11:1),"2,6 Dimethylheptanoyl carnitine",ACar(13:0),Ethyl dodecanoate,Styrene,"3,4,5-Trimethoxycinnamic acid",(+)-1-Methylpropyl 3-(methylthio)-2-propenyl disulfide,Perfluorooctanesulfonic acid,Solasodine,Fe,Se
0,1.0,1.0,0.5,0.263682,0.387117,0.328151,0.020939,0.231371,0.051902,0.217438,0.122771,0.106264,0.816216,0.590583,0.066647,0.786134,0.051525,0.239868,0.252520,0.029142
1,1.0,0.0,0.5,0.194030,0.439252,0.743044,0.002588,0.272726,0.028854,0.119486,0.023862,0.043533,0.734026,0.554731,0.006866,0.150046,0.029358,0.108672,0.239337,0.051359
2,1.0,1.0,0.5,0.169154,0.586865,0.713008,0.011719,0.229975,0.040281,0.125097,0.040502,0.036600,0.675493,0.530477,0.009723,0.129669,0.013565,0.914164,0.246706,0.030194
3,0.0,0.0,1.0,0.114428,0.912970,0.701342,0.003589,0.164908,0.001857,0.089222,0.022774,0.015883,0.792372,1.000000,0.004042,0.291497,0.017668,0.064532,0.174317,0.039949
4,0.0,0.0,0.0,0.099502,0.288151,0.326397,0.010207,1.000000,0.016174,0.143089,0.042985,0.023646,0.751979,0.458182,0.009846,0.483494,0.007723,0.153326,0.485919,0.072570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1.0,1.0,0.0,0.711443,0.244867,0.094064,0.295577,0.033711,0.781156,1.000000,0.740256,0.954058,0.102986,0.175659,0.452592,0.033293,0.355111,0.000000,0.430282,0.257768
97,1.0,1.0,0.5,0.303483,0.381483,0.046465,0.004511,0.096918,0.101102,0.121078,0.188922,0.097184,0.373233,0.000000,0.002453,0.108421,0.034376,0.095399,0.599770,0.697413
98,1.0,1.0,0.0,0.482587,0.171565,0.350424,0.011771,0.095793,0.015318,0.097629,0.068039,0.031624,0.333216,0.211343,0.032351,0.100753,0.044418,0.025212,0.810709,0.233196
99,1.0,0.0,0.0,0.537313,0.262593,0.079917,0.012609,0.111698,0.029331,0.099838,0.017900,0.036687,0.333912,0.282328,0.042862,0.117919,0.049774,0.024243,0.991561,0.643146


In [34]:
classifiers = {"KNN": KNeighborsClassifier(n_neighbors=3),
               "Logistic Regression": LogisticRegression(C=1, max_iter=50000),
               "RBF SVM": SVC(C=10, gamma=0.1, max_iter=50000, probability=True),
               "Extra Trees": ExtraTreesClassifier(n_estimators=8),
               "Random Forest": RandomForestClassifier(n_estimators=8),
               "Neural Net": MLPClassifier(alpha=0.001, hidden_layer_sizes=10, learning_rate_init=0.01)
               }

X_train, X_test, y_train, y_test = train_test_split(data, group, test_size=0.20, random_state=42)
print(len(y_test))

for model_name, model in classifiers.items():

    clf = model.fit(X_train, y_train)
   #  print(clf.score(X_test, y_test))

    clf = model.fit(data, group)
   #  print(clf.score(data, group))
   #  explainer = shap.KernelExplainer(clf.predict_proba, X_test)
    # explainer = shap.KernelExplainer(clf.predict_proba, data)

    # shap_values = explainer.shap_values(X_test)
    # shap_values2 = explainer(X_test) 

    # shap.summary_plot(shap_values, X_test, max_display=10, plot_type="bar", plot_size=(12,8), title=model_name, show=False)
    explainer = shap.Explainer(clf.predict,data)
    shap_values = explainer(data)

    shap.plots.beeswarm(shap_values, order=shap_values.abs.max(0), max_display=6, show=False)
    # shap.plots.bar(shap_values)
    directory = "./shap_results/"
    if not os.path.exists(directory):
        os.makedirs(directory)

    plt.savefig("./shap_results/" + model_name+"_dot.pdf", format='pdf', dpi=1000, bbox_inches='tight')

    # for i in range(len(X_test)):
    #     shap.plots.bar(shap_values[i])

    #     plt.savefig("./shap_results/" + model_name+"_bar_%d.pdf" %i, format='pdf', dpi=1000, bbox_inches='tight')
    #     plt.close()

    # temp_df = pd.DataFrame(np.abs(shap_values[0]), columns=X_test.columns)
    # temp_df.to_csv("./shap_results/" + model_name+"_bar.csv", index=False)
    # np.savetxt("./shap_results/" + model_name+"_bar.csv", shap_values[0], delimiter=',', header=X_test.columns.values.tolist())

    plt.close()


21


Permutation explainer: 102it [01:34,  1.03s/it]                         
